In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, log_loss,roc_auc_score

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/sevilla hackathon/Data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/sevilla hackathon/Data/test.csv')
features=train.drop(['rating_num'], axis=1)
label=train['rating_num']
features.head(5)

,row_id,scout_id,winner,team,competitionId,player_position_1,player_position_2,player_height,player_weight,player_general_var_0,...,team2_defensive_derived_var_10,team2_offensive_derived_var_11,team2_offensive_derived_var_12,team2_offensive_derived_var_13,team2_offensive_derived_var_14,team2_defensive_derived_var_15,team2_offensive_derived_var_16,team2_defensive_derived_var_17,team2_offensive_derived_var_18,team2_offensive_derived_var_19
0,1,13,winner,team1,8,7.0,7.0,0.317073,0.48,0.0,...,0.193658,0.131548,0.097131,0.600944,0.338892,0.347980,0.034157,0.288962,0.310727,0.204925
1,3,16,loser,team2,8,3.0,9.0,0.463415,0.42,0.0,...,0.190500,0.173697,0.272801,0.605659,0.526761,0.468976,0.054034,0.334962,0.380073,0.295392
2,4,4,loser,team1,5,11.0,11.0,0.682927,0.44,0.0,...,0.312314,0.250756,0.212631,0.558833,0.686523,0.441749,0.219441,0.414288,0.570757,0.437500
3,5,13,loser,team2,4,17.0,17.0,0.682927,0.58,0.0,...,0.282556,0.155969,0.206503,0.533002,0.373959,0.527733,0.095201,0.423040,0.330754,0.235558
4,7,11,draw,team1,5,10.0,10.0,0.731707,0.70,0.0,...,0.450453,0.199050,0.134002,0.688773,0.681724,0.733625,0.128719,0.448360,0.444946,0.312500


In [ ]:
def binary_encode(df, column, positive_value):
    df = df.copy()
    df[column] = df[column].apply(lambda x: 1 if x == positive_value else 0)
    return df

In [ ]:
def three_encode(df, column, a_value,b_value):
    df = df.copy()
    df[column] = df[column].apply(lambda x: 2 if x == a_value else (1 if x == b_value
                                  else 0))
    return df

In [ ]:
features= three_encode(features, 'winner', 'winner','draw')
features=binary_encode(features, "team","team1")
features.head(5)

,row_id,scout_id,winner,team,competitionId,player_position_1,player_position_2,player_height,player_weight,player_general_var_0,...,team2_defensive_derived_var_10,team2_offensive_derived_var_11,team2_offensive_derived_var_12,team2_offensive_derived_var_13,team2_offensive_derived_var_14,team2_defensive_derived_var_15,team2_offensive_derived_var_16,team2_defensive_derived_var_17,team2_offensive_derived_var_18,team2_offensive_derived_var_19
0,1,13,2,1,8,7.0,7.0,0.317073,0.48,0.0,...,0.193658,0.131548,0.097131,0.600944,0.338892,0.347980,0.034157,0.288962,0.310727,0.204925
1,3,16,0,0,8,3.0,9.0,0.463415,0.42,0.0,...,0.190500,0.173697,0.272801,0.605659,0.526761,0.468976,0.054034,0.334962,0.380073,0.295392
2,4,4,0,1,5,11.0,11.0,0.682927,0.44,0.0,...,0.312314,0.250756,0.212631,0.558833,0.686523,0.441749,0.219441,0.414288,0.570757,0.437500
3,5,13,0,0,4,17.0,17.0,0.682927,0.58,0.0,...,0.282556,0.155969,0.206503,0.533002,0.373959,0.527733,0.095201,0.423040,0.330754,0.235558
4,7,11,1,1,5,10.0,10.0,0.731707,0.70,0.0,...,0.450453,0.199050,0.134002,0.688773,0.681724,0.733625,0.128719,0.448360,0.444946,0.312500


In [ ]:
test= three_encode(test, 'winner', 'winner','draw')
test=binary_encode(test, "team","team1")
test.head(5)

,row_id,scout_id,winner,team,competitionId,player_position_1,player_position_2,player_height,player_weight,player_general_var_0,...,team2_defensive_derived_var_10,team2_offensive_derived_var_11,team2_offensive_derived_var_12,team2_offensive_derived_var_13,team2_offensive_derived_var_14,team2_defensive_derived_var_15,team2_offensive_derived_var_16,team2_defensive_derived_var_17,team2_offensive_derived_var_18,team2_offensive_derived_var_19
0,14785,16,1,1,7.0,4.0,4.0,0.317073,0.44,0.0,...,0.268842,0.190509,0.311739,0.628642,0.405323,0.309039,0.114849,0.454992,0.304133,0.328816
1,15125,2,1,1,9.0,11.0,11.0,0.560976,0.60,0.0,...,0.346640,0.198072,0.246104,0.536949,0.486006,0.487637,0.105279,0.501402,0.381214,0.499620
2,13095,16,0,0,10.0,19.0,19.0,0.195122,0.30,0.0,...,0.351175,0.177065,0.262470,0.650741,0.567063,0.486774,0.062118,0.251021,0.586227,0.267774
3,6079,16,0,1,11.0,3.0,9.0,0.341463,0.30,0.0,...,0.496021,0.313902,0.405065,0.490855,0.591251,0.608131,0.294538,0.322264,0.685016,0.293783
4,16584,11,2,1,5.0,4.0,4.0,0.585366,0.70,0.2,...,0.396398,0.191068,0.042083,0.838704,0.689723,0.522480,0.090986,0.437002,0.476399,0.437500


In [ ]:
def rmissingvaluecol(dff,threshold):
    l = []
    l = list(dff.drop(dff.loc[:,list((100*(dff.isnull().sum()/len(dff.index))>=threshold))].columns, 1).columns.values)
    print("# Columns having more than %s percent missing values:"%threshold,(dff.shape[1] - len(l)))
    print("Columns:\n",list(set(list((dff.columns.values))) - set(l)))
    return l

l=rmissingvaluecol(features,80)
df = features[l]

# Columns having more than 80 percent missing values: 51
Columns:
 ['player_other_ratio_var_14', 'player_other_ratio_var_32', 'player_other_ratio_var_37', 'team2_other_ratio_var_33', 'player_other_ratio_var_24', 'player_other_ratio_var_33', 'team1_other_raw_var_43', 'team2_other_raw_var_51', 'player_other_ratio_var_34', 'team1_other_raw_var_19', 'team2_other_raw_var_72', 'player_offensive_ratio_var_72', 'player_offensive_ratio_var_70', 'team1_other_raw_var_38', 'player_offensive_ratio_var_69', 'player_offensive_ratio_var_68', 'player_other_ratio_var_28', 'player_other_ratio_var_23', 'team1_other_raw_var_40', 'team2_other_raw_var_38', 'player_other_ratio_var_27', 'player_other_ratio_var_67', 'team1_other_raw_var_92', 'team1_other_raw_var_50', 'team2_other_raw_var_43', 'team1_other_raw_var_51', 'player_other_ratio_var_26', 'player_other_ratio_var_25', 'player_positional_ratio_var_22', 'team2_other_raw_var_82', 'player_defensive_ratio_var_38', 'player_defensive_ratio_var_39', 'team2_other

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
def missing(dff):
    print (round((dff.isnull().sum() * 100/ len(dff)),2).sort_values(ascending=False))
missing(df)

player_offensive_ratio_var_13    79.61
player_other_ratio_var_66        74.45
player_offensive_ratio_var_40    74.21
player_offensive_ratio_var_19    61.60
player_offensive_ratio_var_57    60.99
                                 ...  
player_general_var_2              0.00
player_general_var_3              0.00
player_general_var_5              0.00
scout_id                          0.00
row_id                            0.00
Length: 748, dtype: float64


In [ ]:
df.apply(lambda x: x.fillna(x.median(),inplace=True)) 

row_id                            None
scout_id                          None
winner                            None
team                              None
competitionId                     None
                                  ... 
team2_defensive_derived_var_15    None
team2_offensive_derived_var_16    None
team2_defensive_derived_var_17    None
team2_offensive_derived_var_18    None
team2_offensive_derived_var_19    None
Length: 748, dtype: object

In [ ]:
#Check for  null values
df.isnull().sum()

row_id                            0
scout_id                          0
winner                            0
team                              0
competitionId                     0
                                 ..
team2_defensive_derived_var_15    0
team2_offensive_derived_var_16    0
team2_defensive_derived_var_17    0
team2_offensive_derived_var_18    0
team2_offensive_derived_var_19    0
Length: 748, dtype: int64

In [ ]:
test=test[l]
test.shape

(8774, 748)

In [ ]:
#df= train.to_numpy()
# split data into input and output columns
#X, y = df[:, :-1], df[:, -1]

In [ ]:
# creating testing and training datasets
#spliting the data in to test and train sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, label, test_size = 0.3)
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

In [ ]:
!pip uninstall scikit-learn -y

!pip install -U scikit-learn

Found existing installation: scikit-learn 1.0.2
Uninstalling scikit-learn-1.0.2:
  Successfully uninstalled scikit-learn-1.0.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.8 MB 58.7 MB/s 


In [ ]:
from sklearn.feature_selection import SelectKBest, chi2,f_regression
# configure to select all features
fs = SelectKBest(score_func=f_regression, k='all')
# learn relationship from training data
fs.fit(X_train, y_train)
# transform train input data
X_train_fs = fs.transform(X_train)
# transform test input data
X_test_fs = fs.transform(X_test)
#print(X_train_fs, X_test_fs, fs)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [ ]:
# what are scores for the features
for i in range(len(fs.scores_)):
	print('Feature %d: %f' % (i, fs.scores_[i]))
# plot the scores
from matplotlib import pyplot
pyplot.bar([i for i in range(len(fs.scores_))], fs.scores_)
pyplot.show()

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2,mutual_info_regression
...
# feature selection
def select_features(X_train, y_train, X_test):
	# configure to select all features
	fs = SelectKBest(score_func=mutual_info_regression, k='all')
	# learn relationship from training data
	fs.fit(X_train, y_train)
	# transform train input data
	X_train_fs = fs.transform(X_train)
	# transform test input data
	X_test_fs = fs.transform(X_test)
	return X_train_fs, X_test_fs, fs

# feature selection
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test)
# what are scores for the features
for i in range(len(fs.scores_)):
	print('Feature %d: %f' % (i, fs.scores_[i]))
# plot the scores
pyplot.bar([i for i in range(len(fs.scores_))], fs.scores_)
pyplot.show()

CatBoost Regression

In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 76 kB/s 


In [ ]:
pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 564 kB 33.1 MB/s 


In [ ]:
import catboost as cb
import shap
#from matplotlib import pyplot as pltfrom sklearn.datasets  
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance

In [ ]:
train_dataset = cb.Pool(X_train, y_train) 
test_dataset = cb.Pool(X_test, y_test)

In [ ]:
model = cb.CatBoostRegressor(loss_function='RMSE')

In [ ]:
grid = {'iterations': [100, 150],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}
model.grid_search(grid, train_dataset)

Streaming output truncated to the last 5000 lines.
91:	learn: 1.6304057	test: 1.7147250	best: 1.7147250 (91)	total: 5.67s	remaining: 3.58s
92:	learn: 1.6293807	test: 1.7145537	best: 1.7145537 (92)	total: 5.73s	remaining: 3.51s
93:	learn: 1.6282030	test: 1.7140908	best: 1.7140908 (93)	total: 5.79s	remaining: 3.45s
94:	learn: 1.6268328	test: 1.7136388	best: 1.7136388 (94)	total: 5.84s	remaining: 3.38s
95:	learn: 1.6254300	test: 1.7130984	best: 1.7130984 (95)	total: 5.9s	remaining: 3.32s
96:	learn: 1.6240144	test: 1.7119159	best: 1.7119159 (96)	total: 5.96s	remaining: 3.26s
97:	learn: 1.6229315	test: 1.7110056	best: 1.7110056 (97)	total: 6.01s	remaining: 3.19s
98:	learn: 1.6218344	test: 1.7106333	best: 1.7106333 (98)	total: 6.07s	remaining: 3.13s
99:	learn: 1.6204309	test: 1.7102296	best: 1.7102296 (99)	total: 6.14s	remaining: 3.07s
100:	learn: 1.6186690	test: 1.7090140	best: 1.7090140 (100)	total: 6.21s	remaining: 3.01s
101:	learn: 1.6170729	test: 1.7078030	best: 1.7078030 (101)	total: 6

{'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
               47,
               48,
               49,
             

In [ ]:
pred = model.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, pred)))
r2 = r2_score(y_test, pred)
print("Testing performance")
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2))

Testing performance
RMSE: 1.55
R2: 0.28


In [ ]:
y_pred = model.predict(test[l])
y_pred

array([7.08791148, 5.6379303 , 6.46425622, ..., 6.54363449, 5.77165771,
       7.19366684])

In [ ]:
result = pd.DataFrame(y_pred,columns=['rating_num'], )
result['row_id']=test['row_id']
result=result[['row_id','rating_num']]
result

,row_id,rating_num
0,14785,7.087911
1,15125,5.637930
2,13095,6.464256
3,6079,5.742138
4,16584,7.300252
...,...,...
8769,21259,5.714501
8770,9371,7.916776
8771,23226,6.543634
8772,26623,5.771658


In [ ]:
def round_nearest(x, a):
    return round(x / a) * a

In [ ]:
result['rating_num']=round_nearest(result['rating_num'],0.5)
result

,row_id,rating_num
0,14785,7.0
1,15125,5.5
2,13095,6.5
3,6079,5.5
4,16584,7.5
...,...,...
8769,21259,5.5
8770,9371,8.0
8771,23226,6.5
8772,26623,6.0


In [ ]:
result.to_csv('/content/drive/MyDrive/sevilla hackathon/AH/solution1.csv',index=False)